**Muchas partes de este cuaderno pueden hacerse a mano, copiar archivos a la carpeta, cortar audios etc etc, aqui ya meteriamos audios tras normalizacion demux y diarizacion**

In [ ]:
SP_WORKDIR = "/content/drive/MyDrive/rodserling-sampleuploads/" #@param {type:"string"}
SP_EXT = "wav"

In [ ]:
#@title
%cd /content
!sudo apt install sox
!git clone https://github.com/openai/whisper.git
!pip install git+https://github.com/openai/whisper.git


**Debes tener el audio cortado en segmentos, puede hacerlo con Audacity o con Sox en le siguiente codigo**

In [ ]:
SP_SCRIPT=
#!/bin/bash
EXT='"""+SP_EXT+"""'
WORK_DIR='"""+SP_WORKDIR+"""'
rename 's/\.([^.]+)$/.\L$1/' "$WORK_DIR"/*
cd "$WORK_DIR"
mkdir backup-presplit
mv *.$EXT backup-presplit
cd backup-presplit
rm -rf splits
mkdir splits
for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress silence 1 0.2 0.1% 1 0.2 0.1% : newfile : restart ; done
cd splits
mkdir resplit
for FILE in *.wav; do sox "$FILE" resplit/"$FILE" --show-progress trim 0 9 : newfile : restart ; done
cd resplit
find . -name "*.wav" -type f -size -35k -delete
mv *.wav $WORK_DIR
cd $WORK_DIR
rm -rf splits
ls -al
with open('/content/drive/MyDrive/script-split-samples.sh', 'w') as file:
  #file.write(SP_SCRIPT)

In [ ]:
!sh /content/drive/MyDrive/script-split-samples.sh

In [ ]:
NEWDATASET="vctk-rodserling-22k-ds"
NEWSPEAKER="rodserling"
DATADIR=SP_WORKDIR
%cd /content/drive/MyDrive/rodserling-sampleuploads/
!mkdir -p "/content/drive/MyDrive/"$NEWDATASET"/wav48_silence_trimmed/"$NEWSPEAKER
!ls "/content/drive/MyDrive/"$NEWDATASET"/wav48_silence_trimmed/"$NEWSPEAKER
!find . -name '*.wav' -maxdepth 1 -exec bash -c 'for f; do ffmpeg -i "$f" -ar 22050 -ac 1 -sample_fmt s16 -c:a flac "${f%.wav}"_mic1.flac ; done' _ {} +
!mv "/content/drive/MyDrive/"$DATADIR"/"*.flac "/content/drive/MyDrive/"$NEWDATASET"/wav48_silence_trimmed/"$NEWSPEAKER

In [ ]:
!ls /content/drive/MyDrive/$ds_name/wav48_silence_trimmed

In [ ]:
whisper_model = "large-v2" #@param ["large-v2", "large-v1", "medium.en", "small.en", "base.en

In [ ]:
#@title
import whisper
import os, os.path
import glob
import pandas as pd

from pathlib import Path


#model = whisper.load_model("medium.en")
model = whisper.load_model(whisper_model

In [ ]:
newspeakername = "mickey" #@param {type:"string"}

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
options = dict(language="english", beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)

#@title
wavs = '/content/drive/MyDrive/'+ds_name+'/wav48_silence_trimmed/'+newspeakername

paths = glob.glob(os.path.join(wavs, '*.flac'))
print(len(paths))
all_filenames = []
transcript_text = []
try:
	os.mkdir('/content/drive/MyDrive/'+ds_name+'/txt/')
except OSError as error:
	print(error)
try:
	os.mkdir('/content/drive/MyDrive/'+ds_name+'/txt/'+newspeakername+'/')
except OSError as error:
	print(error)

for filepath in paths:
	base = os.path.basename(filepath)
	all_filenames.append(base)
	result = model.transcribe(filepath, **options)
	output = result["text"].lstrip()
	output = output.replace("\n","")
	print(output)
	thefile = str(os.path.basename(filepath).lstrip(".")).rsplit(".")[0]
	thefile = thefile[:-5]
	print(thefile)
	outfilepath = '/content/drive/MyDrive/'+ds_name+'/txt/'+newspeakername+'/'+thefile+'.txt'
	with open(outfilepath, 'w', encoding='utf-8') as indfile:
		indfile.write(output

**Ver si algun fichero se ha quedao en blancoi o si ha fallado algo**

In [ ]:
check_datasetbase = "vits-vctk-narrator-22k-ds/" #@param {type:"string"}

check_speakername = "marge" #@param {type:"string"}

**Mover los audios no transcritos a una carpeta donde la red neuraonal no los vaya a utlizar**




In [ ]:
or_datasetbase = "vits-vctk-narrator-22k-ds/" #@param {type:"string"}

or_speakername = "marge" #@param {type:"string"